## MS20_Task5 _BarryBot_Client (Py3)

<p> This simple client is provided for testing the MS20_Task5_Server (Py3) and the MS20_Task5_SP_Client_Py3. It tries out some of the commands and prints out the responses that are received from the server. After doing this automatically, it allows a user to enter commands in a text window directly from the keyboard. This will allow you to send messages to and receive messages from the SP_client.  And it will also be useful for testing your Android Application when you have completed it.   
Feel free to make changes to any of this Py3 software. 
When debugging any changes, you will probably need to restart the kernel each time you run the Server and this client. After restarting the Kernel, wait until you see a little blue "Kernal Ready" flag before running the Server or this client again. As for the Server, the code is simple, far from optimal but hopefully easy to understand. </p>


In [ ]:
# BarryBot Client.py for Task 5  24 March
import socket
import select
import time
import numpy as np
from scipy.io import wavfile
from IPython import display as Dp
import PIL as pil
from matplotlib import pyplot as plt
%matplotlib inline

# Function to convert a bitstring BS (0100000 ...) to a text string ("ABCDE")
def bitstring_to_string(BS):
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    BA = np.uint8(BBA) - 48
    mySt = np.packbits(BA)
    mySt = mySt.tobytes()
    myString = mySt.decode('latin1')    # (use latin1 instead of utf-8)
    return myString
# Function to convert a bitstring to an np.int16 array.
def bitstring_to_numpyint16array(BS):
    # Convert bit-string to bit-bytearray:
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    BA=np.uint8(BBA) - 48    # Convert to bitarray
    NA=np.packbits(BA)
    NA=NA.tobytes()
    NA=np.frombuffer(NA, dtype=np.int16)
    NA=NA.byteswap()
    return NA
# Function to convert bitstring (BS) to np.uint8 array (NA)
def bitstring_to_numpyuint8array(BS):
    # Convert bit-string to bit-bytearray:
    BBA = bytearray(BS, "latin-1")  # bit-bytearray
    # Convert BBA to BA:
    BA = np.uint8(BBA) - 48
    NA=np.packbits(BA)
    NA=NA.tobytes()
    NA=np.frombuffer(NA, dtype=np.uint8)
    return NA

def send_to_server(msg):
    if len(msg) > 1024: print("send-message length = ", len(msg) )
    S.sendall(bytes(msg+"\n", "utf-8"))         #  Send adds newline on end of msg

def receive_from_server(wait):
    # Receives long messages and time-out (returning zero message) after wait seconds
    Svr_msg = b""  ;  MTU =8192;
    waiting = select.select([S],[],[],wait) # Waits for wait seconds to see if there is msg on Svr 
    if waiting[0]:
        while 1:
            Svr_msg += S.recv(MTU)       # receive (MTU = buffer size)
            if Svr_msg[-1] == 10 : break   #  10 is code for "\n"
            # end of while clause
    # end of if clause
    Svr_msg = Svr_msg.decode("utf-8")
    return Svr_msg.rstrip("\n")   # Return remove newline from end of Svr_msg

def receive_from_keyboard():
    KB_msg = input("Enter a command: ")
    return KB_msg

def to_Print(msgtoS):
    if (len(msgtoS) < 200):
        print("To Svr: ",msgtoS)  
    else: print("To Svr: ",msgtoS[0:200],"...etc... (length=",len(msgtoS),")")

def from_Print(msgfromS):
    L=len(msgfromS)
    if (L < 200): print("From Svr: ",msgfromS)  
    else:
        msg = msgfromS[0:200]+"..etc.."+msgfromS[L-8:L]
        print("From Svr: ",msg,"(length=",L,")")
        
def bschoose(msg):
    print("bschoose not implemented yet on BarryBot")
def lschoose(msg):
     print("lschoose not implemented yet on BarryBot")

def lslisten(msgfromS):
    msgs = msgfromS.split(" ",3)
    fromwhom = msgs[1] ; what = msgs[2] 
    SS=msgs[3].rstrip("\n") ;
    if what=="T":
        Text = SS
        print("received text: ",Text)
        return
    if what == "S":
        mybytes = SS.encode("latin-1")
        speech = np.frombuffer(mybytes, dtype=np.int16)
        print("Received speech:")
        Dp.display(Dp.Audio(speech,rate=8000))
        return
    if what=="I":
        mybytes = SS.encode("latin-1")
        Pic = np.frombuffer(mybytes, dtype=np.uint8)
        img_array = np.reshape(Pic,(256,256,3))
        print("Received image:")
        plt.imshow(img_array)
        plt.show()
        return 
    if what == "J":
        mybytes = SS.encode("latin-1")
        NA = np.frombuffer(mybytes, dtype=np.uint8)
        OFO = open("outfile.jpg","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close
        try:
            img = pil.Image.open("outfile.jpg")
        except: 
            print("Image could not be opened")
            return
        img_array = np.array(img)
        print("Received image")
        try:
            plt.imshow(img_array)
        except:
            print("Image could not be displayed") 
            return
        plt.show()
        return                             
    print("what is not T,S, I, J in lslisten")
    # All other messages are ignored by mylisten
    return # (I know its not needed)

def bslisten(msgfromS):
    msgs = msgfromS.split(" ",3)
    fromwhom = msgs[1] ; what = msgs[2] 
    BS = msgs[3]
    if what=="T":
        Text = bitstring_to_string(BS)
        print("received text: ",Text)
        return
    if what == "S":
        speech = bitstring_to_numpyint16array(BS)
        print("Received speech:")
        Dp.display(Dp.Audio(speech,rate=8000))
        return
    if what=="I":
        Pic = bitstring_to_numpyuint8array(BS)
        img_array = np.reshape(Pic,(256,256,3))
        print("Received image:")
        plt.imshow(img_array)
        plt.show()
        return 
    if what == "J":
        NA = bitstring_to_numpyuint8array(BS)
        OFO = open("outfile.jpg","wb")
        y = bytes(NA)
        OFO.write(y)
        OFO.close
        try:
            img = pil.Image.open("outfile.jpg")
        except: 
            print("Image could not be opened")
            return
        img_array = np.array(img)
        print("Received image")
        try:
            plt.imshow(img_array)
        except:
            print("Image could not be displayed")
            return
        plt.show()
        return                           
    print("what is not T,S, I, J in mylisten")
    # All other messages are ignored by mylisten
    return # (I know its not needed)        
             
S = socket.socket(socket.AF_INET,socket.SOCK_STREAM)    # AF_INET= IPv4  & SOCK_STREAM =  TCP
S.connect(('localhost', 9999))
# We changed S.connect((socket.gethostname(), 9999)) to S.connect(('localhost', 9999))
msgtoS = "Message to server from BarryBot Client"; print("To Svr: ",msgtoS)
send_to_server(msgtoS)        
msg = receive_from_server(0.1);  print("From Svr:",msg)
msgtoS = "REGISTER BarryBot" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS)  
msg = receive_from_server(0.1) ;  print("From Svr:",msg)
msgtoS = "WHO";  print("To Svr: ",msgtoS) ; send_to_server(msgtoS)   
msg = receive_from_server(0.1) ;  print("From Svr:",msg)
msgtoS = "INVITE Dummy1" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS)
msg = receive_from_server(0.1) ; print("From Svr:",msg)
msgtoS = "MSG Dummy1 BarryBot is alive and well" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS) 
msg = receive_from_server(0.1) ; print("From Svr:",msg)
msgtoS = "UNINVITE Dummy1" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS) 
msg = receive_from_server(0.1) ; print("From Svr:",msg)



# Setting the error probability
msgtoS =  "BSPARAMS 0.000005 0.05 0 3200" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS) 
msg = receive_from_server(2) ; print("From Svr:",msg)

# Sending an Image

msgtoS = "MSG Dummy1 BarryBot is alive and well" ; print("To Svr: ",msgtoS) ; send_to_server(msgtoS) 
msg = receive_from_server(0.1) ; print("From Svr:",msg)





KB_msg = ""
while KB_msg != "QUIT":
    KB_msg=receive_from_keyboard()
    msgtoS=KB_msg[:]  # Makes a COPY of the string contents
    if KB_msg[0:5] =="BSMSG": msgtoS=bschoose(KB_msg)
    if KB_msg[0:5] =="LSMSG": msgtoS=lschoose(KB_msg)                   
    to_Print(msgtoS)
    send_to_server(msgtoS) 
    wait = len(msgtoS)/10000.0 
    if wait < 10.0: wait=10.0
    msgfromS = receive_from_server(wait)
    if msgfromS != "": 
        from_Print(msgfromS)
    else: print("No reply from Svr in ",wait," seconds")
    if msgfromS[0:9] =="BSMSGFROM": bslisten(msgfromS)
    if msgfromS[0:9] =="LSMSGFROM": lslisten(msgfromS)
# end of while clause
time.sleep(1.0) # Wait 1 s before closing the socket
S.close()  # close the socket
print("Client terminates")


To Svr:  Message to server from BarryBot Client
From Svr: Message not understood
To Svr:  REGISTER BarryBot
From Svr: Error: clientuserName already in use.  Choose another
To Svr:  WHO
From Svr: Your username must be registered
To Svr:  INVITE Dummy1
From Svr: Your userName  is not registered
To Svr:  MSG Dummy1 BarryBot is alive and well
From Svr:  is not registered - message not sent to Svr
To Svr:  UNINVITE Dummy1
From Svr: Your client username is not registered
To Svr:  BSPARAMS 0.000005 0.05 0 3200
From Svr: Error: Your username must be registered
